<img src="imgs/header.png">

## Visualization techniques for scalar fields in VTK + Python
## Goals 
* Inspect VTK Objects via the qtconsole for Jupyter (using the magic `%qtconsole`)
* Including a new filter, mapper, and actor to visualize the complete/partial mesh 
* Computing new data arrays using the `vtkCalculator()` 
* Performing data transformations: from rectilinear grid to unstructured grid and image data
* Data filtering
* Visualizing scalar fields using, points, surfaces, isosurfaces, and volume rendering 
    * Basics of transfer functions
    

# Challenge 1: Adding a new Filter+Mapper+Actor to visualize the grid 
Try to find out how to visualize the mesh structure (grid). Take a look at [RectilinearGrid.py](http://www.vtk.org/Wiki/VTK/Examples/Python/RectilinearGrid/vtkRectilinearGrid) example from the VTK wiki.

In [1]:
import vtk
#help(vtk.vtkRectilinearGridReader())

rectGridReader = vtk.vtkRectilinearGridReader()
rectGridReader.SetFileName("data/jet4_0.500.vtk")
# do not forget to call "Update()" at the end of the reader
rectGridReader.Update()

In [2]:
rectGridOutline = vtk.vtkRectilinearGridOutlineFilter()
rectGridOutline.SetInputData(rectGridReader.GetOutput())

# New vtkRectilinearGridGeometryFilter() goes here:
rectGridFilter = vtk.vtkRectilinearGridGeometryFilter() 
rectGridFilter.SetInputData(rectGridReader.GetOutput())
# 
#

rectGridOutlineMapper = vtk.vtkPolyDataMapper()
rectGridOutlineMapper.SetInputConnection(rectGridOutline.GetOutputPort())

rectGridGeomMapper = vtk.vtkPolyDataMapper()
rectGridGeomMapper.SetInputConnection(rectGridFilter.GetOutputPort()) # added line

outlineActor = vtk.vtkActor()
outlineActor.SetMapper(rectGridOutlineMapper)
outlineActor.GetProperty().SetColor(0, 0, 3) # changed the color to blue

gridGeomActor = vtk.vtkActor()
gridGeomActor.SetMapper(rectGridGeomMapper)
# Find out how to visualize this as a wireframe 
gridGeomActor.GetProperty().SetRepresentationToWireframe()
# Play with the options you get for setting up actor properties (color, opacity, etc.)

gridGeomActor.GetProperty().SetColor(0, 0, 0) # changed the color to a red 
gridGeomActor.GetProperty().SetOpacity(0.5);



# Create the usual rendering stuff.

renderer = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(renderer)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)
 
renderer.AddActor(outlineActor)
renderer.AddActor(gridGeomActor)
renderer.SetBackground(0, 0, 0)
renderer.ResetCamera()
renderer.GetActiveCamera().Elevation(60.0)
renderer.GetActiveCamera().Azimuth(30.0)
renderer.GetActiveCamera().Zoom(1.0)
 
renWin.SetSize(300, 300)
 
# interact with data uncomment this line to see partial result
#renWin.Render()
#iren.Start()


#### Result
I set the Rectilinear grid to a red color to prove and see that it covered the whole outline (in blue)
<img src="./hw_4_challenge1.png">


# Challenge 2: Using the vtkCalulator to compute the vector magnitude
As you should have noticed our data set has only one point data array named vectors. We need now to use this array to calculate the magnitude of the vectors at each point of the grid. We will do this by using the [vtk.vtkArrayCalculator()](http://www.vtk.org/doc/nightly/html/classvtkArrayCalculator.html).

In [3]:
#%qtconsole

magnitudeCalcFilter = vtk.vtkArrayCalculator()
magnitudeCalcFilter.SetInputConnection(rectGridReader.GetOutputPort())
magnitudeCalcFilter.AddVectorArrayName('vectors')
# Set up here the array that is going to be used for the computation ('vectors')
magnitudeCalcFilter.SetResultArrayName('magnitude')
magnitudeCalcFilter.SetFunction("mag(vectors)")
# Set up here the function that calculates the magnitude of a vector
magnitudeCalcFilter.Update()

#Inspect the output of the calculator using the IPython console to verify the result

#### Result
<img src="./hw_4_challenge2.png">

# Challenge 3: Visualize the data set as colored points based on the "magnitude" value
Take a look at the [`subset`](http://www.vtk.org/doc/nightly/html/classvtkMaskPoints.html) object. What happens when you change the SetOnRadio parameter? Try out changing between different `RandomModeType` options. What does the `SetScalarModeToUsePointData()` function does?

In [4]:
#Extract the data from the result of the vtkCalculator
points = vtk.vtkPoints()
grid = magnitudeCalcFilter.GetOutput()
grid.GetPoints(points)
scalars = grid.GetPointData().GetArray('magnitude')

#Create an unstructured grid that will contain the points and scalars data
ugrid = vtk.vtkUnstructuredGrid()
ugrid.SetPoints(points)
ugrid.GetPointData().SetScalars(scalars)

#Populate the cells in the unstructured grid using the output of the vtkCalculator
for i in range (0, grid.GetNumberOfCells()):
    cell = grid.GetCell(i)
    ugrid.InsertNextCell(cell.GetCellType(), cell.GetPointIds())

In [5]:
#There are too many points, let's filter the points
subset = vtk.vtkMaskPoints()
subset.SetOnRatio(5)
subset.RandomModeOn()
subset.SetRandomModeType(1)
subset.SetInputData(ugrid)

#Make a vtkPolyData with a vertex on each point.
pointsGlyph = vtk.vtkVertexGlyphFilter()
pointsGlyph.SetInputConnection(subset.GetOutputPort())
#pointsGlyph.SetInputData(ugrid)
pointsGlyph.Update()

pointsMapper = vtk.vtkPolyDataMapper()
pointsMapper.SetInputConnection(pointsGlyph.GetOutputPort())
pointsMapper.SetScalarModeToUsePointData()

pointsActor = vtk.vtkActor()
pointsActor.SetMapper(pointsMapper)
renderer.AddActor(pointsActor)

# interact with data uncomment this lines to see partial result
#renWin.Render()
#iren.Start()

#### Result
<img src="./hw_4_challenge3.png">
#### Questions
**What happens when you change the SetOnRadio parameter?**

The result in the image above is when we call the function with a parameter = 50. If we call it with a value of 500, we get the following result:


<img src="./hw_4_challenge3_q1-with500.png">

Now, lets try to reduce the parameter to a value of 5. The result is the following:

<img src="./hw_4_challenge3_q1-with5.png">

In conclusion, the smaller the value for the function, the better we can visualize our scalar field. Lets take another view at that visualization from another angle:

<img src="./hw_4_challenge3_q1-with5v2.png">

**Try out changing between different `RandomModeType` options.**

This function allows us to change the the random selection of the points using three possible options. The first one is the default one, so I would not show it again. I will try the next two.

According to the docs:

 1 - random sample: create a statistically random sample using Vitter's incremental algorithm D without A described in Vitter "Faster Mthods for Random Sampling", Communications of the ACM Volume 27, Issue 7, 1984 (OnRatio and Offset are ignored) O(sample size) 
 
 <img src="./hw_4_challenge3_q2-with1.png">
 
 2 - spatially stratified random sample: create a spatially stratified random sample using the first method described in Woodring et al. "In-situ Sampling of a Large-Scale Particle Simulation for Interactive Visualization and Analysis", Computer Graphics Forum, 2011 (EuroVis 2011). (OnRatio and Offset are ignored) O(N log N)
 
 <img src="./hw_4_challenge3_q2-with2.png">

**What does the `SetScalarModeToUsePointData()` function does?**

Refering to the docs: 

Control how the filter works with scalar point data and cell attribute data.

By default (ScalarModeToDefault), the filter will use point data, and if no point data is available, then cell data is used. Alternatively you can explicitly set the filter to use point data (ScalarModeToUsePointData) or cell data (ScalarModeToUseCellData).

This means that this functions allows us to associate the points with the scalars, in order to build the scalar field we then visualize.

# Challenge 4: Visualize the data set as isosurfaces based on the "magnitude" value
Go to the documentation of vtkContourFilter and explain what does the GenerateValues() do?  Inspect the scalarRange of the magnitude array. What happens when you change these values in the function?

#### Result

Here you can appreciate the visualization form multiple angle

<img src="./hw_4_challenge4-1.png">
<img src="./hw_4_challenge4-2.png">
<img src="./hw_4_challenge4-3.png">
<img src="./hw_4_challenge4-4.png">
<img src="./hw_4_challenge4-5.png">

#### Questions

** Go to the documentation of vtkContourFilter and explain what does the GenerateValues() do? **

Generate numContours equally spaced contour values between specified range. You can increase or decrease the number.
Be careful when you do that, a really big number could make the filter take too long to compute. As an exaple, this took 20 seconds to render in a 8 GB RAM quadcore core-i7 processor: 

<img src="./hw_4_challenge4-isolines100-1.png">
<img src="./hw_4_challenge4-isolines100-2.png">
<img src="./hw_4_challenge4-isolines100-3.png">

** Inspect the scalarRange of the magnitude array. What happens when you change these values in the function? **

We omit some part of the range, therefore, we are missing some information since there are vectors whose magnitude is otuside the specified range. For a new newScalarRange = (5.7123318, 9.7623123), we get the following result:


<img src="./hw_4_challenge4-newrange.png">

In [6]:
scalarRange = ugrid.GetPointData().GetScalars().GetRange()
print(scalarRange)



isoFilter = vtk.vtkContourFilter()
isoFilter.SetInputData(ugrid)
isoFilter.GenerateValues(10, scalarRange) 

# you may change scalarRange for newScalarRange to see the result
#newScalarRange = (5.7123318, 9.7623123)
#isoFilter.GenerateValues(10, newScalarRange) 


isoMapper = vtk.vtkPolyDataMapper()
isoMapper.SetInputConnection(isoFilter.GetOutputPort())

isoActor = vtk.vtkActor()
isoActor.SetMapper(isoMapper)
isoActor.GetProperty().SetOpacity(0.5)

(0.0035992244084406, 13.617075196155374)


# An alternative to define colormaps 

In [7]:
subset = vtk.vtkMaskPoints()
subset.SetOnRatio(10)
subset.RandomModeOn()
subset.SetInputConnection(rectGridReader.GetOutputPort())

#vtk.vtkColorTransferFunction()
#vtk.vtkLookupTable()
lut = vtk.vtkLookupTable()
lut.SetNumberOfColors(256)
lut.SetHueRange(0.667, 0.0)
lut.SetVectorModeToMagnitude()
lut.Build()

hh = vtk.vtkHedgeHog()
hh.SetInputConnection(subset.GetOutputPort())
hh.SetScaleFactor(0.001)

hhm = vtk.vtkPolyDataMapper()
hhm.SetInputConnection(hh.GetOutputPort())
hhm.SetLookupTable(lut)
hhm.SetScalarVisibility(True)
hhm.SetScalarModeToUsePointFieldData()
hhm.SelectColorArray('vectors')
hhm.SetScalarRange((rectGridReader.GetOutput().GetPointData().GetVectors().GetRange(-1)))

hha = vtk.vtkActor()
hha.SetMapper(hhm)

### Renderer, render window, and interactor 

In [8]:
#Option 1: Default vtk render window
renderer = vtk.vtkRenderer()
renderer.SetBackground(0.5, 0.5, 0.5)
renderer.AddActor(outlineActor)
renderer.AddActor(isoActor)
renderer.ResetCamera()

renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
renderWindow.SetSize(500, 500)
renderWindow.Render()

iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renderWindow)
iren.Start()